# UFO Question 

Our data science team has predicted that the Earth is going to be invaded by an alien force in the
next years. Our only hope is to replicate a device that can block all alien technology in a radius of
~300km. Sadly, the device was sold in 2004 to an anonymous buyer to protect her hometown and
we don't know how contact her again. We know that the device has been active since 2004 in one
city in the USA, and we want to know where to start our search.
We've included a dataset called ​ufo.csv​. This dataset contains over 80,000 reports of UFO sightings
over the last century (all of them verified by the ESA). Using this dataset, try to guess the city in
which the device has been hidden.


In [1]:
from pymongo import MongoClient

import pandas as pd

import re
import folium

In [2]:
full_df = pd.read_csv('ufo.csv')
full_df

,Unnamed: 0,datetime,city,state,country,shape,duration,total_time,comments,date_posted,latitude,longitude,year,distance
0,0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700.0,45 minutes,This event took place in early fall around 194...,4/27/2004,29.883056,-97.941111,2004,1242.667772
1,1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200.0,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.384210,-98.581082,2005,1325.486319
2,2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20.0,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.200000,-2.916667,2008,6515.416577
3,3,10/10/1956 21:00,edna,tx,us,circle,20.0,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.978333,-96.645833,2004,1211.971352
4,4,10/10/1960 20:00,kaneohe,hi,us,light,900.0,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.418056,-157.803611,2004,6960.923396
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78504,80326,9/9/2013 21:00,woodstock,ga,us,sphere,20.0,20 seconds,Driving 575 at 21:00 hrs saw a white and green...,9/30/2013,34.101389,-84.519444,2013,318.719195
78505,80328,9/9/2013 22:00,boise,id,us,circle,1200.0,20 minutes,Boise&#44 ID&#44 spherical&#44 20 min&#44 10 r...,9/30/2013,43.613611,-116.202500,2013,2620.021539
78506,80329,9/9/2013 22:00,napa,ca,us,other,1200.0,hour,Napa UFO&#44,9/30/2013,38.297222,-122.284444,2013,3126.991417
78507,80330,9/9/2013 22:20,vienna,va,us,circle,5.0,5 seconds,Saw a five gold lit cicular craft moving fastl...,9/30/2013,38.901111,-77.265556,2013,904.740389


In [3]:
# df solo con avistamientos de EE. UU.

df = full_df.copy()
df= df[df['country'] == 'us']
sel_cols = ['datetime', 'city', 'date_posted', 'latitude', 'longitude', 'year', 'distance']
df = df[sel_cols]
df['geo_loc'] = df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)

In [25]:
df

,datetime,city,date_posted,latitude,longitude,year,distance,geo_loc
0,10/10/1949 20:30,san marcos,4/27/2004,29.883056,-97.941111,2004,1242.667772,"[-97.9411111, 29.8830556]"
3,10/10/1956 21:00,edna,1/17/2004,28.978333,-96.645833,2004,1211.971352,"[-96.6458333, 28.9783333]"
4,10/10/1960 20:00,kaneohe,1/22/2004,21.418056,-157.803611,2004,6960.923396,"[-157.8036111, 21.4180556]"
5,10/10/1961 19:00,bristol,4/27/2007,36.595000,-82.188889,2007,427.334113,"[-82.1888889, 36.595]"
7,10/10/1965 23:45,norwalk,10/2/1999,41.117500,-73.408333,1999,1295.827792,"[-73.4083333, 41.1175]"
...,...,...,...,...,...,...,...,...
78504,9/9/2013 21:00,woodstock,9/30/2013,34.101389,-84.519444,2013,318.719195,"[-84.5194444, 34.1013889]"
78505,9/9/2013 22:00,boise,9/30/2013,43.613611,-116.202500,2013,2620.021539,"[-116.2025, 43.6136111]"
78506,9/9/2013 22:00,napa,9/30/2013,38.297222,-122.284444,2013,3126.991417,"[-122.2844444, 38.2972222]"
78507,9/9/2013 22:20,vienna,9/30/2013,38.901111,-77.265556,2013,904.740389,"[-77.2655556, 38.9011111]"


In [4]:
# conexion con Mongo

cliente=MongoClient()

db=cliente.ufo_db
db.ufo_col

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ufo_db'), 'ufo_col')

In [5]:
db.ufo_col.insert_many(df.to_dict('records'))

In [6]:
db.ufo_col.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [55]:
bef2004 = df[(df['year'] < 2004)]['city'].value_counts()
aft2004 = df[(df['year'] > 2004)]['city'].value_counts()

b2004vc = df[(df['year'] < 2004) & (df.distance <= 300)]['city'].value_counts()

#diff = aft2004 - bef2004
#diff.sort_values(ascending = False)


def extract_index(b2004vc, i):
    try: 
        return b2004vc[i]
    except:
        return 0


    
bef2004_rev =  [extract_index(b2004vc, i) for i in aft2004.index.tolist()]

In [49]:
df2 = pd.DataFrame(data = aft2004)
df2['diff2004'] = bef2004_rev

df2.diff2004.value_counts()
df2 = df2[df2['diff2004'] > 5].sort_values(by = 'diff2004', ascending = False)

# Find coordinates of each city

df2['latitude'] = [df[df.city == cit].iloc[0, -1][1] for cit in df2.index.tolist()]
df2['longitude'] = [df[df.city == cit].iloc[0, -1][0] for cit in df2.index.tolist()]

df2['city'] = df2.index.tolist()
df2

,city,diff2004,latitude,longitude
nashville,nashville,29,36.165833,-86.784444
louisville,louisville,27,38.254167,-85.759444
knoxville,knoxville,13,35.960556,-83.920833
lexington,lexington,11,35.823889,-80.253611
evansville,evansville,11,37.974722,-87.555833
birmingham,birmingham,11,33.520556,-86.802500
huntsville,huntsville,10,34.730278,-86.586111
bowling green,bowling green,8,41.374722,-83.651389
franklin,franklin,6,43.444167,-71.647778
rome,rome,6,43.212778,-75.456111


In [50]:
def geomap(df, zoom_start=1.5):

    df.index=range(len(df)) 

    mapa=folium.Map(location=[0, 0],
                tiles='openstreetmap', zoom_start=zoom_start)

    for i in range(1, len(df)):

        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])], popup=str(df.city[i]), 
                             radius=1, icon=folium.Icon()).add_to(mapa)

    #mapa.save('images/geomap.html')
    
    
    return mapa

In [51]:
map_ = geomap(df2)

In [52]:
# Mapa final con los candidatos
map_